In [1]:
import tensorflow as tf
import tflearn as tfl

import os

from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

CLASS_NUM = 2
DROPOUT = 0.5

PATCH_SIZE = 500

CHANNEL = 3

PIXEL_BLACK=0
PIXEL_WHITE=255

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/home/xutiancheng/anaconda2/envs/ipykernel_py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def alexnet():
    
    # Building 'AlexNet'
    network = input_data(shape=[None, 227, 227, 3])
    
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 2, activation='softmax')
    
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    
    model = tfl.DNN(network, checkpoint_path='.',
                    max_checkpoints=0, tensorboard_verbose=3,
                    tensorboard_dir='/tmp/finetune_alexnet/tensorboard/')
     
    return network, model

In [3]:
train_file = '../0506samples.txt'
val_file = '../0506samplesval.txt'

assert(os.path.isfile(train_file))
assert(os.path.isfile(val_file))

In [4]:
with open(train_file, "r") as fp:
    datasets_=fp.readlines()

datasets_ = [i.split('\n')[0] for i in datasets_]
datasets_

X = [i.split(' ')[0] for i in datasets_]
Y = [int(i.split(' ')[1]) for i in datasets_]

In [5]:
X_0 = tfl.data_utils.ImagePreloader(X, image_shape=(227, 227), \
                                    normalize=True, grayscale=False)
Y_0 = tfl.data_utils.LabelPreloader(Y, n_class=2, categorical_label=True)

In [6]:
len(X_0), len(Y_0)

(10000, 10000)

In [ ]:
tf.reset_default_graph()
network, model = alexnet()

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [ ]:
tfl.init_graph(seed=42, log_device=True, \
               gpu_memory_fraction=0.8)

model.fit(X_0, Y_0, n_epoch=10, shuffle=True, \
          validation_set=None, show_metric=True, \
          batch_size=32, run_id='tflearn_comp1')


Training Step: 970  | total loss: 0.39038 | time: 81.960s
| Momentum | epoch: 004 | loss: 0.39038 - acc: 0.8340 -- iter: 00992/10000
